In [1]:
import sys
sys.path.insert(0, '../src/')
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz
import graph_statistics
import utils
from evaluation import (Evaluation, tabular_from_statistics, df_from_tabular, compute_original_statistics,
                        boxplot)

## Statistics used

In [20]:
statistic_fns = {#'Assortativity':graph_statistics.assortativity,
                 #'Average Degree':graph_statistics.average_degree,
                 'Claw Count':graph_statistics.claw_count,
                 #'Clustering Coefficient':graph_statistics.clustering_coefficient,
                 #'Characteristic Path Length':graph_statistics.compute_cpl,
                 #'Edge Distribution Entropy':graph_statistics.edge_distribution_entropy,
                 #'Gini':graph_statistics.gini,
                 #'LCC Size':graph_statistics.LCC,
                 #'Max Degree':graph_statistics.max_degree,
                 #'Min Degree':graph_statistics.min_degree,
                 #'Num Connected Components':graph_statistics.num_connected_components,
                 #'Power Law α':graph_statistics.power_law_alpha,
                 #'Spectral Gap':graph_statistics.spectral_gap,
                 'Square Count':graph_statistics.square_count,
                 #'Triangle Count':graph_statistics.triangle_count,
                 'Wedge Count':graph_statistics.wedge_count,
                 }

## Get experimental results from our method and NetGAN

In [7]:
eval_ours = Evaluation(experiment_root='../logs/CORA-ML/Ours/',
                            statistic_fns=statistic_fns)
eval_ours.compute_statistics()
eval_ours.aggregate_statistics(num_bins=10)

In [10]:
tabular = tabular_from_statistics(EO_criterion=0.5,
                                  statistics={'ours': eval_ours.mean_std,
                                             })
df = df_from_tabular(tabular)
df

,Claw Count,Square Count,Wedge Count,Edge Overlap (%),ROC-AUC Score,Average Precision,Time (s)
ours,1859835.800 ± 128934.342,6868.600 ± 366.330,83464.200 ± 1742.566,0.551 ± 0.004,0.926 ± 0.002,0.935 ± 0.002,20.556 ± 0.174


In [11]:
eval_ours.statistics

{'Claw Count': array([[  45974.,   83778., 1755254., 1087249., 1762613., 1807230.,
         2335581., 2393197., 2130766., 2364090., 2547962., 2824024.,
         2938911., 2670077., 3089910., 3085597., 3007058., 2495282.,
         2887332., 3042938.],
        [  46507.,   90408.,  970918., 1363535., 1909279., 1622551.,
         1993116., 2002694., 2481094., 2474820., 2314088., 2382211.,
         2927179., 2658101., 2735031., 2444507., 3085963., 2883485.,
         2905835., 3339176.],
        [  46723.,   92039., 1668973.,  924527., 1837776., 2046223.,
         2219875., 2494250., 2696660., 3073395., 2932666., 2717537.,
         2881657., 2980803., 2805197., 3099282., 3069641., 2701680.,
         2816224., 2894900.],
        [  47252.,  100661.,  850976., 2031125., 2079550., 1974163.,
         2692615., 2388238., 2777266., 2487117., 2675857., 2582928.,
         2819734., 2923217., 2904321., 2621104., 3064073., 3159115.,
         3207781., 2789858.],
        [  47427.,   88625., 1460900.,

In [9]:
tabular

({'ours': {'Claw Count': 1859835.8,
   'Square Count': 6868.6,
   'Wedge Count': 83464.2,
   'Edge Overlap (%)': 0.5513489606368864,
   'ROC-AUC Score': 0.9258161066827469,
   'Average Precision': 0.9351323514598684,
   'Time (s)': 20.55571575164795}},
 {'ours': {'Claw Count': 128934.3422047051,
   'Square Count': 366.32968757664185,
   'Wedge Count': 1742.5663143765864,
   'Edge Overlap (%)': 0.00401197583305197,
   'ROC-AUC Score': 0.002404861712708148,
   'Average Precision': 0.0021598144900592453,
   'Time (s)': 0.17387772150782785}})

In [56]:
def make_direct_comparison(datasets, models, statistic_fns, overlap, original_graphs):
    """ Make a table/ heatmap that compares the relative error of two models at a specified edge overlap 
    for a list of datasets and a list of statistics. Always computes error of first model minus
    error of second model.
    Parameters
    ----------
    datasets: List of strings (names of datasets)
    models: Dictionary. Keys are model names, values are +1/-1 to indicate how to merge both relative errors
    statistic_fns: Dictionary. Keys are statistic names, values are functions used to compute the statistics.
    overlap: Float. Consider first graphs of each trial that achieves this overlap.
    original_graphs: Dictionary. Keys are datasets, values are corresponding train graphs.

    Returns
    -------
    comparison_dict. Dictionary. Rows are datasets, columns are graph statistics, cells are linear combination 
                                 of relative errors with the weights from models.values().
    """
    # Create comparison dict and original statistics dict
    relative_error_dict = dict.fromkeys(statistic_fns.keys(), 0)
    comparison_dict = dict.fromkeys(datasets, relative_error_dict)
    original_statistics = dict.fromkeys(datasets, None)
    for model in models.keys():
        for dataset in datasets:
            # Check if original statistic is computed. If not, compute it
            if original_statistics[dataset] is None:
                original_statistics[dataset] = compute_original_statistics(original_graphs[dataset],
                                                                           statistic_fns)
            # Extract statistics for specified model, dataset, and edge overlap
            eval_model_dataset = Evaluation(experiment_root=f'../logs/{dataset}/{model}/',
                                            statistic_fns=statistic_fns)
            eval_model_dataset.compute_statistics()
            _, overlap_statistics = eval_model_dataset.get_specific_overlap_graph(target_overlap=overlap)
            # Compute relative error for all statistics
            for statistic in statistic_fns.keys():
                rel_error = 0
                for trial in overlap_statistics.keys():
                    rel_error += np.abs(overlap_statistics[trial][statistic] - original_statistics[dataset][statistic])
                rel_error /= len(overlap_statistics.keys()) * original_statistics[dataset][statistic]
                comparison_dict[dataset][statistic] += models[model] * np.abs(rel_error)
    df = pd.DataFrame(comparison_dict.values(), comparison_dict.keys())
    
    return comparison_dict

In [66]:
statistic_fns = {#'Assortativity':graph_statistics.assortativity,
                 #'Average Degree':graph_statistics.average_degree,
                 'Claw Count':graph_statistics.claw_count,
                 #'Clustering Coefficient':graph_statistics.clustering_coefficient,
                 #'Characteristic Path Length':graph_statistics.compute_cpl,
                 #'Edge Distribution Entropy':graph_statistics.edge_distribution_entropy,
                 #'Gini':graph_statistics.gini,
                 #'LCC Size':graph_statistics.LCC,
                 #'Max Degree':graph_statistics.max_degree,
                 #'Min Degree':graph_statistics.min_degree,
                 #'Num Connected Components':graph_statistics.num_connected_components,
                 #'Power Law α':graph_statistics.power_law_alpha,
                 #'Spectral Gap':graph_statistics.spectral_gap,
                 'Square Count':graph_statistics.square_count,
                 #'Triangle Count':graph_statistics.triangle_count,
                 'Wedge Count':graph_statistics.wedge_count,
                 }

In [48]:
_A_obs = load_npz('../data/CORA_ML.npz')

val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)
train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))

In [58]:
comparison_dict = make_direct_comparison(datasets=['CORA-ML'], models={'baseline_NetGan':1, 'Ours':-1}, 
                                         statistic_fns=statistic_fns,
                                         overlap=0.5,
                                         original_graphs={'CORA-ML':train_graph})

In [60]:
comparison_dict['NetGAN'] = comparison_dict['CORA-ML']

In [61]:
comparison_dict

{'CORA-ML': {'Assortativity': -0.016877917132620233,
  'Claw Count': -0.03531099576266994,
  'Square Count': 0.1522953462293244,
  'Wedge Count': 0.031208291153547535},
 'NetGAN': {'Assortativity': -0.016877917132620233,
  'Claw Count': -0.03531099576266994,
  'Square Count': 0.1522953462293244,
  'Wedge Count': 0.031208291153547535}}

In [64]:
df = pd.DataFrame(comparison_dict.values(), comparison_dict.keys())

In [67]:
df

,Assortativity,Claw Count,Square Count,Wedge Count
CORA-ML,-0.016878,-0.035311,0.152295,0.031208
NetGAN,-0.016878,-0.035311,0.152295,0.031208


In [71]:
alt.Chart(df).mark_rect()

alt.Chart(...)

In [72]:
source

,x,y,z
0,-5,-5,50
1,-4,-5,41
2,-3,-5,34
3,-2,-5,29
4,-1,-5,26
...,...,...,...
95,0,4,16
96,1,4,17
97,2,4,20
98,3,4,25


In [69]:
import altair as alt
import numpy as np
import pandas as pd

# Compute x^2 + y^2 across a 2D grid
x, y = np.meshgrid(range(-5, 5), range(-5, 5))
z = x ** 2 + y ** 2

# Convert this grid to columnar data expected by Altair
source = pd.DataFrame({'x': x.ravel(),
                     'y': y.ravel(),
                     'z': z.ravel()})

alt.Chart(source).mark_rect().encode(
    x='x:O',
    y='y:O',
    color='z:Q'
)

alt.Chart(...)